# MIDTERM EXAM | DATE: OCTOBER 22, 2025

# Espinosa Revo II / ID:23821150

# Q1

# Prompt (paste verbatim)

# Write a Python function normalize_and_find(text, needle) that trims both ends of text, lowercases it, then returns the index of the first occurrence of needle. Use a safe search that returns -1 when not found. Add a few asserts.

# ChatGPT Code (paste verbatim)

In [87]:
def normalize_and_find(text: str, needle: str) -> int:
    s = text.strip().lower()
    return s.find(needle)

# quick tests
print(normalize_and_find("  Hello World  ", "hello"))
print(normalize_and_find("No match here", "s"))


0
-1


# Critique

# Correctness: Lowers text but does not lower needle, so case-insensitive match may fail (e.g., “World”). Uses find, which safely returns -1. Behaves incorrectly for case-insensitive spec.

# Complexity: strip() and lower() are O(n); find is O(n·m) worst-case with efficient implementations; overall O(n+m) average practical; O(1) extra space (new strings due to immutability).

# Robustness: Works on empty strings. Does not validate types (acceptable per spec). Returning -1 meets “safe search”.

# Readability/Style: Clear and short; could add docstring and lower the needle; add tests for edge cases (empty needle, all-whitespace).

# Faithfulness: Uses strip, lower, find consistent with lecture notes; needs explicit mention of find vs index: index raises ValueError if absent; find returns -1, which avoids exception handling but requires checking -1 downstream.

# Improved Code

In [36]:
def translate(text: str, mapping: dict[str, str]) -> str:
    # simple approach: replace in a loop
    result = text
    for k, v in mapping.items():
        result = result.replace(k, v)
    return result

print(translate("akal!", {"a":"4","e":"3","k":"5"}))


454l!


# Q2

# Prompt (paste verbatim)

# Make translate(text, mapping) that only applies mapping to lowercase letters (leave others unchanged). Be efficient (avoid repeated concatenation). Include quick tests.

# ChatGPT Code (paste verbatim)

In [90]:
def translate(text: str, mapping: dict[str, str]) -> str:
    # simple approach: replace in a loop
    result = text
    for k, v in mapping.items():
        result = result.replace(k, v)
    return result

print(translate("aXe!k", {"a":"4","e":"3","k":"5"}))


4X3!5


# Critique

# Correctness: Applies mapping globally but also transforms uppercase (because .replace("a","4") will not affect uppercase, but it will incorrectly transform occurrences after prior replacements if keys overlap; also spec says only lowercase letters should be considered per-character, not global replace). If mapping has overlapping keys (e.g., “e”->“3” and later something involving “3”), repeated replaces can cascade.

# Complexity: Multiple passes over the string (k passes), O(k·n). Due to immutability, each replace creates a new string, increasing allocations.

# Robustness: Leaves non-lowercase unchanged, but behavior depends on mapping contents; no type checks.

# Readability/Style: Prefer per-character transformation with list accumulation and "".join(...).

# Faithfulness: Our lectures emphasize strings are immutable; show list/join to avoid quadratic behavior.

# Improved Code

In [104]:
from typing import Dict

def translate(text: str, mapping: Dict[str, str]) -> str:
    """Apply mapping only to lowercase letters; others unchanged."""
    out_chars = []
    for ch in text:
        if "a" <= ch <= "z":  # lowercase only
            out_chars.append(mapping.get(ch, ch))
        else:
            out_chars.append(ch)
    return "".join(out_chars)

if __name__ == "__main__":
    m = {"a": "4", "e": "3", "o": "0","k":"5"}
    assert translate("akal!", m) == "454l!"
    assert translate("Hello, world!", m) == "H3ll0, w0rld!"
    assert translate("", m) == ""
    print(translate("aXe!k.",m))


4X3!5.


# Q3

# rompt (paste verbatim)

# Write fmt_money(x) that formats a float as currency with commas and 2 decimals using ties-to-even rounding. Show tests for 2.5 and 3.5.

In [93]:
def fmt_money(x: float) -> str:
    return format(x, ",.2f")

print(fmt_money(1234.5))
print(fmt_money(2.5), fmt_money(3.5))


1,234.50
2.50 3.50


# Critique

# Correctness: format(x, ",.2f") uses Python’s rounding (ties-to-even) after binary float approximation, so values like 2.675 can display as 2.67 due to representation error. For money, prefer Decimal with ROUND_HALF_EVEN.

# Complexity: O(d) to format digits; acceptable. Extra overhead converting to Decimal.

# Robustness: No input validation; acceptable. But recommend converting via Decimal(str(x)) to reduce binary float artifacts when the source is a Python float literal or string input.

# Readability/Style: Add docstring, tests for ties and representation pitfalls.

# Faithfulness: Demonstrates formatted printing and discusses FP error.

# Improved Code

In [107]:
from decimal import Decimal, ROUND_HALF_EVEN, InvalidOperation

def fmt_money(x: float | str) -> str:
    """Format as currency with commas and 2 decimals using ties-to-even rounding."""
    try:
        # Convert via str to avoid binary float artifacts (e.g., 2.675).
        d = Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_HALF_EVEN)
        # Insert commas and 2 decimals
        return f"{d:,.2f}"
    except (InvalidOperation, ValueError) as e:
        raise ValueError(f"Invalid monetary value: {x}") from e


if __name__ == "__main__":   
    assert fmt_money("3.505") == "3.50"   # even hundredths digit -> stays
    assert fmt_money("3.515") == "3.52"   # odd hundredths digit -> rounds up
    assert fmt_money("2.675") == "2.68"   # banker’s rounding example
    assert fmt_money(1234.5) == "1,234.50"
    assert fmt_money(2.5) == "2.50"       # not a tie for 2 decimals, so stays .50
    assert fmt_money(3.5) == "3.50"       # also stays .50

    print(fmt_money(1234.56))
    print(fmt_money(2.5))
    print(fmt_money(3.5))

1,234.56
2.50
3.50


# Q4

# Prompt (paste verbatim)

# Write a CLI program exponent.py that reads two numbers, validates them, and prints a ** b. Handle errors nicely. Prefer try/except over str.isdigit. Include a few tests inside main guard.

# ChatGPT Code (paste verbatim)

In [56]:
def main():
    a = input("Base: ")
    b = input("Exponent: ")
    if a.replace(".", "", 1).isdigit() and b.replace(".", "", 1).isdigit():
        print(float(a) ** float(b))
    else:
        print("Please enter numbers only.")

if __name__ == "__main__":
    main()


Base:  5
Exponent:  2


25.0


# ritique

# Correctness: isdigit()-style check fails for negatives (e.g., “-2”), scientific notation (“1e3”), and leading/trailing spaces. This is exactly why we prefer try/except float(...).

# Complexity: O(len(input)) parsing; trivial.

# Robustness: Poor: rejects valid formats; no exception handling; no messages with context; no exit code differentiation.

# Readability/Style: Add docstrings, clear messages, and structure (helper function for parse).

# Faithfulness: Demonstrates why isdigit() is insufficient.

# Improved Code

In [61]:
import sys

def parse_float(s: str) -> float:
    s = s.strip()
    try:
        return float(s)
    except ValueError as e:
        raise ValueError(f"Invalid number: {s!r}") from e

def main() -> None:
    try:
        a_str = input("Base: ")
        b_str = input("Exponent: ")
        a = parse_float(a_str)
        b = parse_float(b_str)
        result = a ** b
        print(f"{a} ** {b} = {result}")
    except ValueError as err:
        print(f"Error: {err}", file=sys.stderr)

if __name__ == "__main__":
    # Manual/simple smoke tests
    # (Interactive program—tests here are illustrative and won't run automatically)
    # Example valid: 2 and 3 -> 8
    # Example valid: -2 and 3 -> -8
    # Example valid: 9 and 0.5 -> 3.0
    main()


Base:  5
Exponent:  6


5.0 ** 6.0 = 15625.0


# Q5

# Prompt (paste verbatim)

# Implement check_password(pw) returning a dict of rule booleans plus overall. Policies: len>=10, at least one upper, lower, digit, one of !@#$%^&*, and no spaces. Also add an input loop that keeps asking until valid or Q/q.

# ChatGPT Code (paste verbatim)

In [62]:
def check_password(pw: str) -> dict:
    rules = {
        "length": len(pw) >= 10,
        "upper": any(c.isupper() for c in pw),
        "lower": any(c.islower() for c in pw),
        "digit": any(c.isdigit() for c in pw),
        "symbol": any(c in "!@#$%^&*" for c in pw),
        "no_spaces": " " not in pw,
    }
    rules["ok"] = all(rules.values())
    return rules

def prompt_loop():
    while True:
        pw = input("Enter password (Q to quit): ")
        if pw.lower() == "q":
            print("Bye")
            break
        r = check_password(pw)
        print(r)
        if r["ok"]:
            print("Accepted")
            break

if __name__ == "__main__":
    prompt_loop()


Enter password (Q to quit):  q


Bye


# Critique

# Correctness: Meets all rules. Good.

# Complexity: Single pass checks are O(n). Space O(1).

# Robustness: Accepts empty/space-only strings; handled by rules. Could hide dictionary printing and provide friendlier messages per failed rule; could avoid echoing password on-screen in real systems.

# Readability/Style: Clear; add docstrings and formatted feedback.

# Faithfulness: Uses conditionals, any, loop with break, continue is not needed.

# Improved Code

In [111]:
from typing import Dict

ALLOWED_SYMBOLS = set("!@#$%^&*")

def check_password(pw: str) -> Dict[str, bool]:
    """Return policy flags and overall result for a password."""
    flags = {
        "length": len(pw) >= 10,
        "upper": any(c.isupper() for c in pw),
        "lower": any(c.islower() for c in pw),
        "digit": any(c.isdigit() for c in pw),
        "symbol": any(c in ALLOWED_SYMBOLS for c in pw),
        "no_spaces": " " not in pw,
    }
    flags["ok"] = all(flags.values())
    return flags

def prompt_loop() -> None:
    """Prompt until password is valid or user quits with Q/q."""
    while True:
        pw = input("Enter password (Q to quit): ")
        if pw.lower() == "q":
            print("Goodbye.")
            break
        res = check_password(pw)
        if res["ok"]:
            print("Password accepted.")
            break
        else:
            print("Password not accepted. Failed rules:")
            for k, v in res.items():
                if k != "ok" and not v:
                    print(f" - {k}")
            print("Please try again.\n")

if __name__ == "__main__":
    # Simple unit checks
    assert check_password("Aa1!aaaaaa")["ok"] is True
    assert check_password("shortA1!")["ok"] is False
    assert check_password("NoDigits!!!!!")["digit"] is False
    assert check_password("with space !A1")["no_spaces"] is False
    print("Q5 tests passed.")
    # prompt_loop()  # interactive


Q5 tests passed.


# Q6

# Prompt (paste verbatim)

# Write a roll() that returns 1..6 uniformly. Simulate 100,000 trials of two dice and estimate P(sum==7). Print empirical vs exact. Mention seeding for reproducibility.

# ChatGPT Code (paste verbatim)

In [71]:
import random

def roll() -> int:
    return random.randint(1, 6)

def main():
    trials = 100_000
    count = 0
    for _ in range(trials):
        if roll() + roll() == 6:
            count += 1
    print("Empirical:", count / trials)
    print("Exact:", 5/36)

if __name__ == "__main__":
    main()


Empirical: 0.13986
Exact: 0.1388888888888889


# ritique

# Correctness: Correct simulation and exact probability (6 favorable out of 36). Good.

# Complexity: O(trials); memory O(1).

# Robustness: No seeding; add optional random.seed(…) for reproducibility. Could parameterize trials.

# Readability/Style: Add docstrings and format percentages.

# Improved Code

In [77]:
import random
from typing import Tuple

def roll() -> int:
    """Return a fair die outcome in [1, 6]."""
    return random.randint(1, 6)

def simulate(trials: int, seed: int | None = None) -> Tuple[float, float]:
    """Return (empirical_prob_sum7, exact_prob_sum7)."""
    if seed is not None:
        random.seed(seed)
    count = 0
    for _ in range(trials):
        if roll() + roll() == 6:
            count += 1
    empirical = count / trials
    exact = 6 / 36
    return empirical, exact

if __name__ == "__main__":
    emp, ex = simulate(100_000, seed=42)
    print(f"Empirical P(sum==6): {emp:.5f}")
    print(f"Exact P(sum==6):     {ex:.5f}")
    print("Q6 done.")


Empirical P(sum==6): 0.13837
Exact P(sum==6):     0.16667
Q6 done.


# Q7

# Prompt (paste verbatim)

# Create Vehicle base with color, mileage, fuel_liters. Methods: drive(km, km_per_liter) and refuel(liters). Car and Truck inherit; Truck overrides drive to reduce efficiency (e.g., +10% fuel consumption). Include simple tests.

# ChatGPT Code (paste verbatim)

In [82]:
class Vehicle:
    """Base class for all vehicles."""

    def __init__(self, color: str, mileage: float, fuel_liters: float):
        self.color = color
        self.mileage = mileage
        self.fuel_liters = fuel_liters

    def drive(self, km: float, km_per_liter: float) -> float:
        """Drive up to km distance, consuming fuel, never going below 0."""
        if km < 0 or km_per_liter <= 0:
            raise ValueError("Distance and efficiency must be positive.")

        max_possible_km = self.fuel_liters * km_per_liter
        actual_km = min(km, max_possible_km)
        fuel_used = actual_km / km_per_liter
        self.mileage += actual_km
        self.fuel_liters = max(0.0, self.fuel_liters - fuel_used)
        return actual_km

    def refuel(self, liters: float) -> None:
        """Add fuel."""
        if liters < 0:
            raise ValueError("Cannot refuel negative liters.")
        self.fuel_liters += liters


class Car(Vehicle):
    """Car inherits from Vehicle with no changes."""
    pass


class Truck(Vehicle):
    """Truck consumes 10% more fuel (less efficient)."""
    def drive(self, km: float, km_per_liter: float) -> float:
        # reduce efficiency by 10%
        effective_kpl = km_per_liter * 0.9
        return super().drive(km, effective_kpl)


# ---- Minimal Test Script ----
if __name__ == "__main__":
    car = Car("red", 0, 10)
    truck = Truck("blue", 0, 10)

    print("Car drives 100 km at 10 km/L")
    car_drive = car.drive(100, 10)
    print(f" -> Car drove {car_drive} km, fuel left: {car.fuel_liters:.2f} L")

    print("Truck drives 100 km at 10 km/L (10% worse efficiency)")
    truck_drive = truck.drive(100, 10)
    print(f" -> Truck drove {truck_drive} km, fuel left: {truck.fuel_liters:.2f} L")

    car.refuel(5)
    print(f" -> Car refueled, new fuel: {car.fuel_liters:.2f} L")


Car drives 100 km at 10 km/L
 -> Car drove 100 km, fuel left: 0.00 L
Truck drives 100 km at 10 km/L (10% worse efficiency)
 -> Truck drove 90.0 km, fuel left: 0.00 L
 -> Car refueled, new fuel: 5.00 L


# Critique

# Correctness: Fuel update is wrong: subtracts needed = km/km_per_liter even when we cannot drive full km. Should compute fuel consumed based on can_drive, # not requested distance. Truck override: passing km_per_liter*0.9 to super() increases efficiency (since denominator smaller?)—actually giving fewer km # per liter (worse efficiency) is correct, but the base’s bug remains. Also refuel should validate nonnegative input.

# Complexity: O(1) operations per call. Fine.

# Robustness: No input checks; negative liters or negative km could slip through. Keep fuel non-negative and mileage only increased by actual distance.

# Readability/Style: Add docstrings, type hints, and explicit return values.

# Faithfulness: Demonstrates inheritance and an override; fix core logic.

# Improved Code

In [2]:
class Vehicle:
    """Base class for all vehicles."""
    def __init__(self, color: str, mileage: float = 0.0, fuel_liters: float = 0.0):
        self.color = color
        self.mileage = mileage
        self.fuel_liters = fuel_liters

    def refuel(self, liters: float) -> None:
        """Add fuel (liters must be non-negative)."""
        if liters < 0:
            raise ValueError("Liters must be non-negative.")
        self.fuel_liters += liters

    def drive(self, km: float, km_per_liter: float) -> float:
        """
        Drive up to 'km' at the given efficiency.
        Never consume more fuel than available.
        Returns the actual kilometers driven.
        """
        if km < 0:
            raise ValueError("Distance must be non-negative.")
        if km_per_liter <= 0:
            raise ValueError("Efficiency must be positive.")

        max_km_possible = self.fuel_liters * km_per_liter
        actual_km = min(km, max_km_possible)
        fuel_used = actual_km / km_per_liter

        self.mileage += actual_km
        self.fuel_liters -= fuel_used
        return actual_km


class Car(Vehicle):
    """A normal car with standard efficiency."""
    pass


class Truck(Vehicle):
    """A truck that is less fuel-efficient (80% efficiency of a car)."""
    EFFICIENCY_FACTOR = 0.8

    def drive(self, km: float, km_per_liter: float) -> float:
        # Trucks consume more fuel → reduce efficiency
        adjusted_efficiency = km_per_liter * self.EFFICIENCY_FACTOR
        return super().drive(km, adjusted_efficiency)
# --- Usage demo ---
if __name__ == "__main__":
    car = Car("red", mileage=500, fuel_liters=10)
    truck = Truck("blue", mileage=4000, fuel_liters=8)

    print("Car drives:", car.drive(30, km_per_liter=10), "km")
    print("Car fuel left:", round(car.fuel_liters, 2), "L")

    print("Truck drives:", truck.drive(50, km_per_liter=8), "km")
    print("Truck fuel left:", round(truck.fuel_liters, 2), "L")

Car drives: 30 km
Car fuel left: 7.0 L
Truck drives: 50 km
Truck fuel left: 0.19 L


# Q8


# Prompt (paste verbatim)

# Build a Dog class (name, age, breed, bark()) and a Kennel that manages multiple Dogs with add_dog, remove_dog, oldest_dog(), and find_by_breed(breed). Use composition (Kennel has Dogs). Provide minimal tests.

# ChatGPT Code (paste verbatim)

In [94]:
class Dog:
    def __init__(self, name, age, breed):
        self.name = name
        self.age = age
        self.breed = breed
    def bark(self):
        return "Woof!"

class Kennel:
    def __init__(self):
        self.dogs = []
    def add_dog(self, d):
        self.dogs.append(d)
    def remove_dog(self, name):
        self.dogs = [d for d in self.dogs if d.name != name]
    def oldest_dog(self):
        return max(self.dogs, key=lambda d: d.age) if self.dogs else None
    def find_by_breed(self, breed):
        return [d for d in self.dogs if d.breed == breed]

# tests
k = Kennel()
k.add_dog(Dog("Fido", 3, "Beagle"))
k.add_dog(Dog("Rex", 5, "Labrador"))
print([d.name for d in k.find_by_breed("Beagle")])
print(k.oldest_dog().name)


['Fido']
Rex


# Critique

# Correctness: Satisfies methods. Good.

# Complexity: All operations linear in number of dogs; acceptable.

# Robustness: No type checks; remove_dog removes all with matching name—OK but document behavior. Case-sensitive breed match; could normalize.

# Readability/Style: Add type hints, docstrings, and defensive checks.

# Faithfulness: Correctly uses composition (Kennel “has many” Dogs). Inheritance would be incorrect here (Kennel is not a Dog).

# Improved Code

In [95]:
from typing import List, Optional

class Dog:
    """A simple dog."""
    def __init__(self, name: str, age: int, breed: str) -> None:
        if age < 0:
            raise ValueError("age must be nonnegative")
        self.name = name
        self.age = age
        self.breed = breed

    def bark(self) -> str:
        return "Woof!"

class Kennel:
    """Kennel managing a collection of Dog instances (composition)."""
    def __init__(self) -> None:
        self._dogs: List[Dog] = []

    def add_dog(self, dog: Dog) -> None:
        if not isinstance(dog, Dog):
            raise TypeError("Expected a Dog instance")
        self._dogs.append(dog)

    def remove_dog(self, name: str) -> int:
        """Remove all dogs with matching name; return count removed."""
        before = len(self._dogs)
        self._dogs = [d for d in self._dogs if d.name != name]
        return before - len(self._dogs)

    def oldest_dog(self) -> Optional[Dog]:
        return max(self._dogs, key=lambda d: d.age) if self._dogs else None

    def find_by_breed(self, breed: str) -> List[Dog]:
        b = breed.strip().lower()
        return [d for d in self._dogs if d.breed.lower() == b]

if __name__ == "__main__":
    k = Kennel()
    k.add_dog(Dog("Fido", 3, "Beagle"))
    k.add_dog(Dog("Rex", 5, "Labrador"))
    k.add_dog(Dog("Bella", 5, "Beagle"))
    assert [d.name for d in k.find_by_breed("beagle")] == ["Fido", "Bella"]
    assert k.oldest_dog().age == 5
    assert k.remove_dog("Fido") == 1
    print("Q8 tests passed.")


Q8 tests passed.


# Q9

# Prompt (paste verbatim)

# Create a Python package called helpers with two modules:

# string_utils.py that defines a function shout(s) which uppercases a string.

# math_utils.py that defines a function area(l, w) returning l * w.

# ChatGPT Code (paste verbatim)

In [99]:
# --- helpers emulated inline ---

# string_utils module
class string_utils:
    @staticmethod
    def shout(s: str) -> str:
        return s.upper()

# math_utils module
class math_utils:
    @staticmethod
    def area(l: float, w: float) -> float:
        return l * w
        
# --- main program ---
import types
import sys

# Register fake modules in sys.modules so 'import helpers.string_utils' works
helpers = types.ModuleType("helpers")
helpers.string_utils = string_utils
helpers.math_utils = math_utils
sys.modules["helpers"] = helpers
sys.modules["helpers.string_utils"] = string_utils
sys.modules["helpers.math_utils"] = math_utils

# --- now the same imports as before ---
import helpers.string_utils as su
from helpers.math_utils import area

print(su.shout("AYOOOOO!"))
print(area(4, 4))

AYOOOOO!
16


# Critique
# Correctness: The package structure and imports are valid. Functions behave as specified.
# Complexity: O(1) for both functions.
# Robustness: No type hints or input checks (e.g., non‑numeric input to area). __init__.py is empty; fine, though exposing a clean package namespace can help.
# Readability: Minimal comments/docstrings. Function names are clear. Reference: :contentReference[oaicite:15]index=15
# Faithfulness: Demonstrates imports (import ... as ... and from ... import ...), aligning with our modules/namespace lectures.

# IMPROVED CODE 

In [101]:
# --- inline package builder (single notebook cell, no files needed) ---
from __future__ import annotations
import sys
import types
from typing import Callable, Mapping, Any


def register_inline_package(
    package_name: str,
    modules: Mapping[str, Mapping[str, Any]],
) -> None:
    """
    Create an in-memory package and its submodules and register them in sys.modules.
    Example:
        register_inline_package("helpers", {
            "string_utils": {"shout": shout},
            "math_utils": {"area": area},
        })
    Then you can `import helpers.string_utils as su` normally.
    """
    # Make (or reuse) the package shell
    pkg = sys.modules.get(package_name)
    if not isinstance(pkg, types.ModuleType):
        pkg = types.ModuleType(package_name)
        # mark as a package
        pkg.__path__ = []  # type: ignore[attr-defined]
        sys.modules[package_name] = pkg

    # Expose submodules and attributes
    exported = []
    for mod_name, attrs in modules.items():
        fqmn = f"{package_name}.{mod_name}"
        mod = types.ModuleType(fqmn)
        mod.__package__ = package_name

        for attr_name, obj in attrs.items():
            setattr(mod, attr_name, obj)

        sys.modules[fqmn] = mod
        setattr(pkg, mod_name, mod)
        exported.append(mod_name)

    # Optional: make `from helpers import *` reveal submodules
    pkg.__all__ = tuple(exported)


# --- utils implementations (plain functions with validation & docs) ---

# string_utils.py
def shout(s: str) -> str:
    """Return the input string uppercased."""
    if not isinstance(s, str):
        raise TypeError("shout(s): 's' must be a str")
    return s.upper()


def titlecase(s: str) -> str:
    """Return Title Case for the string."""
    if not isinstance(s, str):
        raise TypeError("titlecase(s): 's' must be a str")
    return s.title()


# math_utils.py
def area(l: float, w: float) -> float:
    """Area of a rectangle (l × w). Both must be finite and non-negative."""
    for name, v in (("l", l), ("w", w)):
        if not isinstance(v, (int, float)):
            raise TypeError(f"area({name}): must be int|float")
        if v < 0:
            raise ValueError(f"area({name}): must be non-negative")
    return float(l) * float(w)

def perimeter(l: float, w: float) -> float:
    """Perimeter of a rectangle (2l + 2w)."""
    for name, v in (("l", l), ("w", w)):
        if not isinstance(v, (int, float)):
            raise TypeError(f"perimeter({name}): must be int|float")
        if v < 0:
            raise ValueError(f"perimeter({name}): must be non-negative")
    return 2.0 * (float(l) + float(w))

# --- register the inline package + submodules ---
register_inline_package(
    "helpers",
    {
        "string_utils": {
            "shout": shout,
            "titlecase": titlecase,
        },
        "math_utils": {
            "area": area,
            "perimeter": perimeter,
        },
    },
)

# --- use them with normal imports (exactly like real files) ---
import helpers.string_utils as su
from helpers.math_utils import area, perimeter

print(su.shout("AYOOOOO!"))
print(area(4, 4))
print(perimeter(4, 4))
print(su.titlecase("TRESMARES"))

# NOTES: It defines a register_inline_package function that dynamically registers in-memory modules (like helpers.string_utils and helpers.math_utils) in sys.modules,
# allowing normal import syntax and organized, reusable helper functions for string and math operations.

AYOOOOO!
16.0
16.0
Tresmares


# Q10

# Prompt (paste verbatim)

# Write cel_to_far and far_to_cel with type hints and docstrings. Build a CLI that asks which direction (C2F or F2C), parses input safely, and prints the result. Include tests, including the -40 crossover.


# ChatGPT Code (paste verbatim)

In [102]:
def cel_to_far(c: float) -> float:
    return c * 9/5 + 32

def far_to_cel(f: float) -> float:
    return (f - 32) * 5/9

def main():
    mode = input("C2F or F2C? ").strip().upper()
    if mode == "C2F":
        x = float(input("Celsius: "))
        print(cel_to_far(x))
    elif mode == "F2C":
        x = float(input("Fahrenheit: "))
        print(far_to_cel(x))
    else:
        print("Unknown mode.")

if __name__ == "__main__":
    main()


C2F or F2C?  F2C
Fahrenheit:  38


3.3333333333333335


# Critique
# 
# Correctness: Formulas are correct.

# Time/Space Complexity: O(1).

# # Robustness: No error handling for invalid numbers; no docstrings; minimal formatting; no tests.

# Readability/Style: Add docstrings, type hints, and nice messages.

# Faithfulness: Demonstrates functions and control flow; we’ll add exceptions and tests.

# Improved Code

In [6]:
def cel_to_far(c: float) -> float:
    return c * 9 / 5 + 32

def far_to_cel(f: float) -> float:
    return (f - 32) * 5 / 9

def main():
    mode = input("C2F or F2C? ").strip().upper()
    if mode == "C2F":
        x = float(input("Celsius: "))
        result = cel_to_far(x)
        print(f"{x}°C is equal to {result:.2f}°F")
    elif mode == "F2C":
        x = float(input("Fahrenheit: "))
        result = far_to_cel(x)
        print(f"{x}°F is equal to {result:.2f}°C")
    else:
        print("Unknown mode. Please enter C2F or F2C.")

if __name__ == "__main__":
    main()


C2F or F2C?  C2F
Celsius:  38


38.0°C is equal to 100.40°F
